In [89]:
import numpy as np 
import pandas as pd
import os, sys
import gseapy as gp

### ESTIMATE

#### estimate score

In [3]:
sig = pd.read_excel("/Users/qiluzhou/Desktop/umass/research/score/data/stromal_immune_sig.xlsx", header = 0, sheet_name = 0)
# immune_sig_est = list(sig.loc[2,:])[2:]
# stromal_sig_est = list(sig.loc[1,:])[2:]

/Users/qiluzhou/opt/anaconda3/lib/python3.9/site-packages/openpyxl/worksheet/_read_only.py:79: UserWarning: Unknown extension is not supported and will be removed
  for idx, row in parser.parse():


In [17]:
data = pd.read_table("/Users/qiluzhou/Desktop/umass/research/score/data/chol_tcga.txt", sep = "\t")
# remove NA and duplicate names
data = data.drop_duplicates(subset = ['Hugo_Symbol'])
data = data.dropna(subset = "Hugo_Symbol", axis = 0)
#data.to_csv("/Users/qiluzhou/Desktop/umass/research/score/data/chol_tcga_clean.txt", sep = "\t")

In [35]:
# rank normalization
row_names = data.Hugo_Symbol # gene names
df = data.iloc[:, 2:]

df_ranked = df.apply(pd.to_numeric).rank(axis = 0, method = "average")
df_ranked.index = row_names
df_normalized = 10000*df_ranked/df_ranked.shape[0]
gene_set_names = list(sig.iloc[1:3, 0])
score = np.empty((df_ranked.shape[1], 2))

In [36]:
# ssgsea
for i in range(0,2):
    
    gene_set = sig.iloc[i+1,2:]
    common = list(set(gene_set) & set(row_names))

    if len(common) == 0 : 
        continue

    ES_vector = [0]*df_normalized.shape[1]

    for sample in range(0, df_normalized.shape[1]):
        ordered_genes = df_normalized.iloc[:,sample].sort_values(ascending = False)

        hit_ind = ordered_genes.index.isin(gene_set)
        no_hit_ind = 1- hit_ind

        ordered_genes1 = ordered_genes.pow(1./4)
        hit_exp = ordered_genes1[hit_ind]

        no_hit_penalty = np.cumsum(no_hit_ind/sum(no_hit_ind))

        hit_reward = np.cumsum((hit_ind*ordered_genes1)/sum(hit_exp))

        ES_vector[sample] = sum(hit_reward - no_hit_penalty)
    
    score[:, i] = ES_vector

In [41]:
# get estimate score
score = pd.DataFrame(score)
score.index = data.columns[2:]
score.columns = ["stromal", "immune"]
score['estimate'] = score['stromal'] + score['immune']
# get tumor purity(Affymetrix-only)
purity = np.cos(0.6049872018 + 0.0001467884*score['estimate'])

In [49]:
score.head()

,stromal,immune,estimate
TCGA-3X-AAV9-01,3480.865820,4655.945959,8136.811779
TCGA-3X-AAVA-01,1426.604986,2067.395840,3494.000827
TCGA-3X-AAVB-01,3261.592173,3056.319093,6317.911267
TCGA-3X-AAVC-01,937.550265,2459.131260,3396.681525
TCGA-3X-AAVE-01,1680.669364,4245.019743,5925.689107


#### filter common gene, plot tumor purity, output gct

### ISTME

#### get score

In [144]:
immune_istme=['CD40LG','THEMIS','PYHIN1','TRAT1','FCRL1','SPIB','GPR174','SH2D1A','CCR4','ITK','FCRL3','C5orf20','FIGF','UBASH3A','MS4A1','CD300LG','SAMD3','ADH1B','GZMK','CHRDL1','ABI3BP','FCER2','TIFAB','P2RY12','CLEC10A','RSPO2','PCDH15','HLA-DOA','PLA2G2D','CLEC17A','CD3G','CCL19','PTPRC','C17orf87','GRIA1','CD8A','PRG4','P2RY13','SFTPC','HLA-DPB1','AADAC','EOMES','AOAH','CD1E','CCR2','CCL5','GFRA1','TFEC','CLDN18','FGL2','C4orf7','CD1B','GZMA','HLA-DPA1','SCARA5','PLEK','ZNF683','CD19','HLA-DRA','CD84','PIK3CG','NCKAP1L','CR1','WIF1','CLEC12A','KIAA0408','PIGR','CXCL13','CD74','TLR8','CHIT1','IL7R','HLA-DRB1','HLA-DQA1','COL6A6','LYZ','SFTPA2','CYBB','COL29A1','LCP1','IGJ','UBD','HLA-DQA2','CXCL9','PPYR1','IFNG','MRC1','FAM26F','B2M','CD1A','HLA-B','GBP5','ADAMDEC1','LAPTM5','C1QB','ITGB2','C1QA','SUCNR1','MARCO','HLA-C','VSIG4','F13A1','CXCL10','C1QC','HLA-A','CD163','FCGR3A','TMSL3']
stromal_istme=['MDK','RPL8','S100P','FTH1','GPR87','UBC','CLDN3','FGL1','TMSB10','HSPB1','ACTB','ITPKA','KRT19','S100A2','EEF1A2','PABPC1','RPLP0','HSP90AB1','NMU','KRT8','KRT7','EPCAM','YWHAZ','MYH9','ACTG1','NPW','MMP13','TUBA1B','ERRFI1','DSP','KRT18','P4HB','ENO1','PKM2','RHOV','ALDOA','JUP','LDHA','HMGA1','HSPA1A','MIF','TUBB','HMGB3','CYP24A1','TPI1','FAM83A','RECQL4','FN1','VEGFA','GAPDH','TK1','TUBB3','UBE2C','MYBL2','COL3A1','COL1A1','COL1A2','COL11A1']

In [145]:
istme_data = df.apply(pd.to_numeric)
istme_data.index = list(row_names)
df_ranked1 = istme_data.rank(axis = 0, method = "average")
df_ranked1 = df_ranked1.apply(abs)

In [172]:
def istme(geneset, df):

    ES_vector = [0]*df.shape[1]

    for sample in range(0, df.shape[1]):
        ordered_genes = df.iloc[:,sample].sort_values(ascending = False)

        hit_ind = ordered_genes.index.isin(geneset)
        no_hit_ind = 1- hit_ind

        ordered_genes1 = ordered_genes.pow(1./4)
        hit_exp = ordered_genes1[hit_ind]

        no_hit_penalty = np.cumsum(no_hit_ind/sum(no_hit_ind))
        hit_reward = np.cumsum((hit_ind*ordered_genes1)/sum(hit_exp))

        ES_vector[sample] = sum(hit_reward - no_hit_penalty)

    ES_normalized = [i/(max(ES_vector)-min(ES_vector)) for i in ES_vector]
    score = ES_normalized
    return(score)

In [174]:
istme_immune_score = istme(immune_istme, df_ranked1)
istme_stromal_score = istme(stromal_istme, df_ranked1)
istme = pd.DataFrame({"immune":istme_immune_score, "stromal": istme_stromal_score})
istme.index = df_ranked1.columns

In [188]:
istme.head()

,immune,stromal
TCGA-3X-AAV9-01,0.237001,2.292723
TCGA-3X-AAVA-01,0.029753,1.978521
TCGA-3X-AAVB-01,0.085616,2.146487
TCGA-3X-AAVC-01,0.023140,2.157706
TCGA-3X-AAVE-01,0.258784,2.077586


#### 